In [1]:
from transformers import PreTrainedTokenizerFast

tokenizer = PreTrainedTokenizerFast.from_pretrained("skt/kogpt2-base-v2",
    bos_token='</s>', eos_token='</s>', unk_token='<unk>',
    pad_token='<pad>', mask_token='<mask>') 
tokenizer.tokenize("안녕하세요. 한국어 GPT-2 입니다.😤:)l^o")


Downloading: 100%|██████████| 2.83M/2.83M [00:01<00:00, 2.31MB/s]


['▁안녕',
 '하',
 '세',
 '요.',
 '▁한국어',
 '▁G',
 'P',
 'T',
 '-2',
 '▁입',
 '니다.',
 '😤',
 ':)',
 'l^o']

In [3]:
import torch
from transformers import GPT2LMHeadModel

model = GPT2LMHeadModel.from_pretrained('skt/kogpt2-base-v2')
text = '별 하나의 추억과'
input_ids = tokenizer.encode(text)
gen_ids = model.generate(torch.tensor([input_ids]),
                           max_length=128,
                           repetition_penalty=2.0,
                           pad_token_id=tokenizer.pad_token_id,
                           eos_token_id=tokenizer.eos_token_id,
                           bos_token_id=tokenizer.bos_token_id,
                           use_cache=True)
generated = tokenizer.decode(gen_ids[0,:].tolist())
print(generated)


심세경은 "지난해 12월 말 현재 국내총생산(GDP) 대비 국가채무 비율은 40.9%로 경제협력개발기구(OECD)의 평균인 43.8%보다 높다"며 "이런 상황에서 정부가 재정건전성을 위해 국채 발행을 늘리고 있다"고 지적했다.
그는 이어 “국채발행 증가가 경기부양 효과를 상쇄할 수 있다는 점에서 바람직하다”면서도 “이번 추경 편성은 경제회복을 위한 불가피한 선택”이라고 덧붙였다.
이날 토론회에 참석한 한 참석자는 이 같은 정부의 방침에 대해 ‘재정 건전성 확보’라는 명분을 내세우며 반대 입장을 분명히 했다.
김영주 한국조폐공사 사
